# Module: PICRUSt2

Metabarcoding analyses target a single marker gene, typically as a proxy for studying community composition. However, this approach is inherently limited in assessing the functional potential of a community since it focuses on just one gene. PICRUSt2 addresses this limitation by leveraging reference genomes associated with 16S rRNA sequences in metabarcoding data. These reference genomes are used to predict functional profiles, providing insights into the potential metabolic capabilities of the community.

This notebook outlines the steps involved in running the PICRUSt2 functional prediction pipeline. This notebook includes (1) phylogenetic placement of query reads, (2) prediction of function, (3) prediction of number of 16S genes per representative sequence, (4) inferring metagenome, and (5) inferring pathway abundances.

Before using PICRUSt2, take note of its limitations as described <a href="https://github.com/picrust/picrust2/wiki/Key-Limitations">here</a>.

Created by: _Microbial Oceanography Laboratory (MOLab)_

---
## How to Use This Notebook

1. Make sure tools are installed already (see below if not yet).
2. Activate environment. Replace environment name accordingly.
```bash
conda activate picrust2-env
```
2. Open jupyter notebook with the command below and select the notebook.
```bash
jupyter notebook
```
3. To run the cells in this notebook, press Shift+Enter.

---
## Tools Used
1. **PICRUSt2**

To install, run the command below. This creates an environment and installs the PICRUSt2 package at the same time.
```bash
conda create --name picrust2-env -c bioconda -c conda-forge picrust2
```

<div class="alert alert-block alert-info">
<b>Note:</b> 
    
Some steps are resource-intensive. If it is not feasible for you to run these steps locally, you can access the <a href='https://usegalaxy.eu/'>Galaxy EU webserver</a>.
</div>

<div class="alert alert-block alert-info">
<b>Note:</b> 
    
QIIME2 also offers a plugin to PICRUSt2. Installation procedure for QIIME2 plugin is discussed here: <a href='https://github.com/picrust/picrust2/wiki/q2-picrust2-Tutorial/'>q2 PICRUSt2</a>.
</div>

---
## Starting Files
1. Representative sequences, `rep-seqs.fa` (see **QIIME2 Metabarcoding OTU Clustering Module** or **QIIME2 Metabarcoding Denoising Module**)
2. Feature table, `feature-table.tsv` (see **QIIME2 Metabarcoding OTU Clustering Module** or **QIIME2 Metabarcoding Denoising Module**)

---
## Expected Outputs

1. Feature tables of function (EC, KO) and/or pathway (KEGG pathways or MetaCyc) counts across samples.

---
## Table of Contents
 * [**Step 1: Phylogenetic placement**](#Step-1:-Phylogenetic-placement)  
 * [**Step 2: Hidden-state prediction**](#Step-2:-Hidden-state-prediction)  
 * [**Step 3: Predicting Number of 16S rRNA genes per representative sequence**](#Step-3:-Predicting-Number-of-16S-rRNA-genes-per-representative-sequence)
 * [**Step 4: Predict metagenome of the samples**](#Step-4:-Predict-metagenome-of-the-samples)
 * [**Step 5: Infer pathways**](#Step-5:-Infer-pathways)  
 * [**Step 6: Add descriptions**](#Step-6:-Add-descriptions)  

---
# <font color = 'gray'>Phylogenetic Placement</font>

This step aligns our representative sequences (OTUs or ASVs) to the sequences in the reference database using HMMER. Afterwards, it runs the phylogenetic placement algorithm (default: EPA-NG) to place the query sequences on the reference phylogenetic tree.

If you want to use custom reference files, please refer to this [page](https://github.com/picrust/picrust2/wiki/Sequence-placement).

<div class="alert alert-block alert-warning">
<b>Warning:</b> 

This step is resource-intensive.

| option/input | description |
| :-: | :- |
| `-s` | Query sequences. |
| `-o` | Output tree file. |
| `-p` | Number of processes to run in parallel. |

In [ ]:
!place_seqs.py \
    -s rep-seqs.fa \
    -o 1-out_tree.tre \
    -p 1

---
# <font color = 'gray'>Predicting 16S rRNA Gene Copy</font>

This step calculates nearest-sequenced taxon index (NSTI) and the number of 16S genes per predicted genome. 

The NSTI quantifies how close your query sequences are to its nearest 16S sequence in the reference database. Small NSTI values yields more accurate predictions while larger NSTI values result to less accurate predictions. By default, NSTI values greater than 2 are omitted from the predictions.

Additionally, the 16S copy number per predicted genome is used to normalize the abundance of the genes to be predicted in the next steps.

| option/input | description |
| :-: | :- |
| `-i` | Trait table {options: 16S, COG, EC, KO, PFAM, TIGRFAM, PHENO}. |
| `-t` | Tree file. |
| `-o` | Output table. |
| `--calculate_NSTI` | Enable calculation of NSTI. |

In [ ]:
!hsp.py \
    -i 16S \
    -t 1-out_tree.tre \
    -o 2-out_16S_NSTI_calc.tsv \
    --calculate_NSTI

Let's take a peek at a subset of the output (`2-out_16S_NSTI_calc.tsv`). The table shows the predicted number of 16S genes for every genome that is associated with the representative sequence. Additionally, it also displays the NSTI values.

In [ ]:
!cut -d '	' -f1-5 2-out_16S_NSTI_calc.tsv | head -n 10 | column -t

---
# <font color = 'gray'>Predicting Gene Family Copy Number</font>

This step predicts the abundances of gene families, using Enzyme Commission (EC) system, for each representative sequence with a predicted genome. Alternatively, we could use different pre-calculated count table from the following options: 'EC', '16S', 'COG', 'KO', 'PFAM', 'TIGRFAM', 'PHENO'.

| option/input | description |
| :-: | :- |
| `-i` | Trait table {options: 16S, COG, EC, KO, PFAM, TIGRFAM, PHENO}. |
| `-t` | Tree file. |
| `-o` | Output table. |

In [ ]:
!hsp.py \
    -i EC \
    -t 1-out_tree.tre \
    -o 3-out_EC_prediction.tsv

Let's take a peek at a subset of the output (`3-out_EC_prediction.tsv`). From this table, we can see the predicted abundances of each EC group for each representative sequence.

<div class="alert alert-block alert-warning">
<b>Warning:</b> 

The output below have many columns and may be messy. It may be easier to view it in other softwares (e.g. MS Excel).

In [ ]:
!cut -d '	' -f1-5 1-output-files/2-out_EC_prediction.tsv | head -n 10 | column -t

---
# <font color = 'gray'>Predict Metagenome of the Samples</font>

From the associated genome for each ASV/OTU, we can now generate the metagenome of our samples with this step. By using the previously calculated 16S copies per genome, we infer the number of genomes, and subsequently, calculate the abundance of gene families for the metagenome. Taking into account the gene copy number of the marker gives a better estimate of the functional landscape of the community.

<h4>Detailed explanation to normalization:</h4>

Consider the table below 

| Representative Sequences in Sample 1 | Number of 16S Genes in Metabarcoding Data | Number of 16S Genes per Genome | Number of EC 1.1.1.1 per Genome |  Number of EC 1.1.1.2 per Genome |
| --- | --- | --- | --- | --- |
| OTU1 | 20 | 2 | 5 | 3 |
| OTU2 | 35 | 5 | 3 | 6 |
| OTU3 | 16 | 4 | 4 | 10 |

From the predicted number of 16S genes (generated by Step 3) and the expected number of 16S genes per predicted genome, we can calculate the expected number of genomes for each of our representative sequence.

| Representative Sequences in Sample 1 | Predicted Number of 16S Genes | Number of 16S Genes per Genome | Expected Number of Genome | Number of EC 1.1.1.1 per Genome |  Number of EC 1.1.1.2 per Genome |
| --- | --- | --- | --- | --- | --- |
| OTU1 | 20 | 2 | 20/2 = 10 | 5 | 3 |
| OTU2 | 35 | 5 | 35/5 = 7 | 3 | 6 |
| OTU3 | 16 | 4 | 16/4 = 4 | 4 | 10 |

Finally, we can calculate the number of gene family and generate the functional profile of our metagenome.

| Representative Sequences in Sample 1 | Predicted Number of 16S Genes | Number of 16S Genes per Genome | Expected Number of Genome | Number of EC 1.1.1.1 per Genome |  Number of EC 1.1.1.2 per Genome | Total Number of EC 1.1.1.1 | Total Number of EC 1.1.1.2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| OTU1 | 20 | 2 | 20/2 = 10 | 5 | 3 | 5x10 = 50 | 3x10 = 30 | 
| OTU2 | 35 | 5 | 35/5 = 7 | 3 | 6 | 3x5 = 15 | 6x5 = 30 |
| OTU3 | 16 | 4 | 16/4 = 4 | 4 | 10 | 4x4 = 16 | 10x4 = 40 |

And so, Sample 1 is predicted to have 81 (50+15+16) copies of EC 1.1.1.1 and 110 (30+30+40) copies of EC 1.1.1.2

| option/input | description |
| :-: | :- |
| `-i` | ASV/OTU feature table. |
| `-m` | Table of predicted marker gene copy numbers. |
| `-f` | Table of predicted gene family copy numbers. |
| `-o` | Output directory. |
| `--strat_out` | Output table stratified by sequences as well. |

In [ ]:
!metagenome_pipeline.py \
    -i feature-table.tsv \
    -m 2-out_16S_NSTI_calc.tsv \
    -f 3-out_EC_prediction.tsv \
    -o 4-out_EC_metagenome \
    --strat_out

<b>Output Files:</b>
1. `seqtab_norm.tsv.gz` - Representative sequences' feature table that is normalized by the predicted number of 16S genes
2. `weighted_nsti.tsv.gz` - Table containing weighted NSTI values for each sample
3. `pred_metagenome_unstrat.tsv.gz` - Abundance of each gene family per sample
4. `pred_metagenome_contrib.tsv.gz` - Shows calculations somewhat similar to the tables above

In [ ]:
#Unzip the output file of Step 4
!gunzip 4-out_EC_metagenome/*

Let's take a peek at one of the outputs (`pred_metagenome_contrib.tsv`). This table shows the predicted abundance of each EC group contributed by each representative sequence in each sample.

In [ ]:
!cut -d '	' -f1-8 4-out_EC_metagenome/pred_metagenome_contrib.tsv | head -n 10 | column -t

Meanwhile, the table below summarizes the predicted total abundance of each EC gene family per sample. You could use this table to compare which function(s) are elevated/depressed between samples.

In [ ]:
!cut -d '	' -f1-11 1-output-files/4-out_EC_metagenome/pred_metagenome_unstrat.tsv | head -n 10 | column -t

---
# <font color = 'gray'>Infer Pathways</font>

The output above gives a gene-level (or at least a proxy to genes) abundance across samples. This may sometimes be difficult to analyze due to its high dimension. As an alternative, PICRUSt2 can aggregate these gene-level functions to pathways. Depending on your goal, this can make it easier because gene functions are now summarized at a higher level.

| option/input | description |
| :-: | :- |
| `-i` | Input TSV table of gene family abundances (either the unstratified or stratified output of metagenome_pipeline.py). |
| `-o` | Output folder for pathway abundance output. |

If you are using a different gene family, make sure to specify the filepath for the gene-family-to-pathway mapping file using the `-m` argument. By default, the mapping file used is `metacyc_path2rxn_struc_filt_pro.txt`. Run `pathway_pipeline.py -h` to find the location of other mapping files.

<div class="alert alert-block alert-warning">
<b>Warning:</b> 

KEGG is no longer open-source. The mapping files originally provided by PICRUSt2 for KO to KEGG pathway may now be outdated.

In [ ]:
!pathway_pipeline.py \
    -i 4-out_EC_metagenome/pred_metagenome_contrib.tsv \
    -o 5-out_pathways

Then extract the zipped output.

In [ ]:
#Unzip the output file of Step 5
!gunzip 5-out_pathways/*

The outputs of this step is similar to the previous step (i.e. Predicting the metagenome). However, instead of the abundances of gene families, the predicted pathway abundances are now the ones displayed.

In [ ]:
#Check path_abun_contrib.tsv
!cut -d '	' -f1-8 5-out_pathways/path_abun_contrib.tsv | head -n 10 | column -t

In [ ]:
#Check path_abun_unstrat.tsv
!cut -d '	' -f1-11 5-out_pathways/path_abun_unstrat.tsv | head -n 10 | column -t

---
# <font color = 'gray'>Add descriptions</font>

| option/input | description |
| :-: | :- |
| `-i` | Abundance table. |
| `-m` | Gene family or pathway used in the abundance table {options: METACYC,COG,EC,KO,PFAM,TIGRFAM}.
| `-o` | Output abundance table with descriptions. |

PICRUSt2, by default, only outputs a table with functional IDs presented. To produce a table that is more informative, we could add descriptions (pathway name, EC gene name, etc.) of these functional IDs.

In [ ]:
!add_descriptions.py \
    -i 4-out_EC_metagenome/pred_metagenome_unstrat.tsv \
    -m EC \
    -o 4-out_EC_metagenome/pred_metagenome_unstrat_desc.tsv

In [ ]:
!add_descriptions.py \
    -i 5-out_pathways/path_abun_unstrat.tsv \
    -m METACYC \
    -o 5-out_pathways/path_abun_unstrat_desc.tsv

<div class="alert alert-block alert-info">
<b>Note:</b> 
    
PICRUSt2 does not provide a KEGG pathway-to-description mapping file. If you want to add descriptions for KEGG pathways, run the following commands to generate a custom mapping file. Then use the <code>--custom_map_table</code> argument instead of <code>-m</code>.

<br>
1. Get the KEGG pathway ID to description mapping from API.
    
<code>kegg_pathways=$(curl -# https://rest.kegg.jp/list/pathway)</code>

<br>
2. Replace "map" by "ko" and redirect to a file.

<code>echo -e "${kegg_pathways}" | awk -F'\t' -v OFS='\t' '{gsub(/map/, "ko", $1); print $1,$2}' > kegg_desc_mapping.txt</code>

<br>

You can also find the KEGG mapping file in the same folder as this notebook. Note, however, that since PICRUSt2 no longer uses the updated version of KEGG, some pathway IDs may have empty descriptions.
</div>